<a href="https://colab.research.google.com/github/bwowby/DS/blob/master/ir-basic/word2vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 네이버 영화리뷰 한국어 word2vec 만들기

In [6]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 51.3MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

#### 데이터 텍스트 파일 불러오기

In [9]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fe55669f128>)

In [11]:
#pd.read_table : Read general delimited file into DataFrame.
train_data = pd.read_table("ratings.txt")

In [13]:
train_data.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


#### 전처리 하기

In [19]:
len(train_data)

200000

In [25]:
#any(): 하나라도 true인지 확인
train_data.isnull().values.any() 

True

In [26]:
# Null 값 존재 행 제거 (any: 하나라도 존재하면 삭제 all : 전체가 Na여야 삭제)
train_data = train_data.dropna(how = 'any')
print(train_data.isnull().values.any())

False


In [27]:
# 8개가 줄었음
len(train_data)

199992

In [31]:
# 정규 표현식을 통해 한글 외의 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")

In [32]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 형태소 분석기 OKT를 사용한 토근화 작업(다소 시간 소요)
okt = Okt()
tokenized_data = []
for sentence in train_data['document'] : 
  temp_X = okt.morphs(sentence, stem=True) #토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  tokenized_data.append(temp_X)